### Aim of the notebook is to visualize the receptive fields of convolution filters 

This notebook is simplified version of github repo [https://github.com/metalbubble/cnnvisualizer](https://github.com/metalbubble/cnnvisualizer)

Parts of codes have been used for the ICLR'15 paper Object Detectors Emerge in Deep Scene CNN

In [1]:
import torch
from torch.autograd import Variable as V
from torchvision import transforms as trn
import os
import numpy as np
import cv2
from dataset import Dataset
import torchvision.models as models
import torch.utils.data as data
import torchvision.models as models

/home/saurabh/anaconda3/envs/py36torch/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/saurabh/anaconda3/envs/py36torch/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# visualization setup
img_size = (224, 224)       # input image size
segment_size = (120,120)    # the unit segmentaiton size
num_top = 12                # how many top activated images to extract
margin = 3                  # pixels between two segments
threshold_scale = 0.2       # the scale used to segment the feature map. Smaller the segmentation will be tighter.

In [3]:
# dataset setup
batch_size = 64
num_workers = 6

In [4]:
# Load required model
model_name = 'resnet18_imagenet'
model = models.resnet18(pretrained=True)
features_names = ['layer4']

In [5]:
model.eval()
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [11]:
### Download the image dataset
!sh download_images.sh

--2018-11-28 22:50:22--  http://places.csail.mit.edu/unit_annotation/data/images.tar
Resolving places.csail.mit.edu (places.csail.mit.edu)... 128.30.100.255
Connecting to places.csail.mit.edu (places.csail.mit.edu)|128.30.100.255|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 755445760 (720M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 720,45M  17,8MB/s    in 80s     

2018-11-28 22:51:48 (9,01 MB/s) - ‘images.tar’ saved [755445760/755445760]

images/
images/readme.txt
images/imagenet/
images/imagenet/012988.jpg
images/imagenet/009763.jpg
images/imagenet/005704.jpg
images/imagenet/017503.jpg
images/imagenet/017971.jpg
images/imagenet/020938.jpg
images/imagenet/004144.jpg
images/imagenet/016896.jpg
images/imagenet/008123.jpg
images/imagenet/008684.jpg
images/imagenet/016343.jpg
images/imagenet/024454.jpg
images/imagenet/024826.jpg
images/imagenet/007051.jpg
images/imagenet/019596.jpg
images/imagenet/015983.jpg
ima

images/imagenet/012266.jpg
images/imagenet/017538.jpg
images/imagenet/020903.jpg
images/imagenet/020571.jpg
images/imagenet/005098.jpg
images/imagenet/009758.jpg
images/imagenet/000061.jpg
images/imagenet/020048.jpg
images/imagenet/017001.jpg
images/imagenet/009261.jpg
images/imagenet/000558.jpg
images/imagenet/005206.jpg
images/imagenet/016641.jpg
images/imagenet/008386.jpg
images/imagenet/021608.jpg
images/imagenet/024156.jpg
images/imagenet/004446.jpg
images/imagenet/004834.jpg
images/imagenet/001318.jpg
images/imagenet/008853.jpg
images/imagenet/008421.jpg
images/imagenet/019733.jpg
images/imagenet/015754.jpg
images/imagenet/019094.jpg
images/imagenet/007921.jpg
images/imagenet/007553.jpg
images/imagenet/014114.jpg
images/imagenet/018173.jpg
images/imagenet/006313.jpg
images/imagenet/004780.jpg
images/imagenet/024945.jpg
images/imagenet/024537.jpg
images/imagenet/021269.jpg
images/imagenet/016220.jpg
images/imagenet/001779.jpg
images/imagenet/004027.jpg
images/imagenet/024290.jpg
i

images/imagenet/016864.jpg
images/imagenet/007704.jpg
images/imagenet/010988.jpg
images/imagenet/019564.jpg
images/imagenet/019916.jpg
images/imagenet/022938.jpg
images/imagenet/015971.jpg
images/imagenet/015503.jpg
images/imagenet/014896.jpg
images/imagenet/018483.jpg
images/imagenet/006144.jpg
images/imagenet/014343.jpg
images/imagenet/018324.jpg
images/imagenet/004270.jpg
images/imagenet/008217.jpg
images/imagenet/021799.jpg
images/imagenet/001289.jpg
images/imagenet/016077.jpg
images/imagenet/013729.jpg
images/imagenet/024760.jpg
images/imagenet/017190.jpg
images/imagenet/009457.jpg
images/imagenet/009825.jpg
images/imagenet/005842.jpg
images/imagenet/005430.jpg
images/imagenet/012169.jpg
images/imagenet/017637.jpg
images/imagenet/005397.jpg
images/imagenet/014085.jpg
images/imagenet/006525.jpg
images/imagenet/006957.jpg
images/imagenet/014722.jpg
images/imagenet/018745.jpg
images/imagenet/006282.jpg
images/imagenet/007365.jpg
images/imagenet/002849.jpg
images/imagenet/019105.jpg
i

images/imagenet/008288.jpg
images/imagenet/005308.jpg
images/imagenet/000824.jpg
images/imagenet/000456.jpg
images/imagenet/020146.jpg
images/imagenet/012651.jpg
images/imagenet/003543.jpg
images/imagenet/003931.jpg
images/imagenet/023053.jpg
images/imagenet/011744.jpg
images/imagenet/002303.jpg
images/imagenet/010104.jpg
images/imagenet/022613.jpg
images/imagenet/021448.jpg
images/imagenet/016401.jpg
images/imagenet/016873.jpg
images/imagenet/024316.jpg
images/imagenet/008661.jpg
images/imagenet/004606.jpg
images/imagenet/001158.jpg
images/imagenet/017241.jpg
images/imagenet/020208.jpg
images/imagenet/009786.jpg
images/imagenet/000718.jpg
images/imagenet/005046.jpg
images/imagenet/009021.jpg
images/imagenet/017994.jpg
images/imagenet/018333.jpg
images/imagenet/011878.jpg
images/imagenet/014354.jpg
images/imagenet/018494.jpg
images/imagenet/006153.jpg
images/imagenet/014881.jpg
images/imagenet/015966.jpg
images/imagenet/015514.jpg
images/imagenet/019573.jpg
images/imagenet/019901.jpg
i

images/imagenet/002707.jpg
images/imagenet/007059.jpg
images/imagenet/019239.jpg
images/imagenet/010972.jpg
images/imagenet/010500.jpg
images/imagenet/022217.jpg
images/imagenet/011895.jpg
images/imagenet/006619.jpg
images/imagenet/003147.jpg
images/imagenet/023825.jpg
images/imagenet/023457.jpg
images/imagenet/011340.jpg
images/imagenet/018479.jpg
images/imagenet/001273.jpg
images/imagenet/008938.jpg
images/imagenet/021763.jpg
images/imagenet/013074.jpg
images/imagenet/012193.jpg
images/imagenet/020684.jpg
images/imagenet/000841.jpg
images/imagenet/000433.jpg
images/imagenet/012634.jpg
images/imagenet/020123.jpg
images/imagenet/000394.jpg
images/imagenet/011086.jpg
images/imagenet/023791.jpg
images/imagenet/003526.jpg
images/imagenet/003954.jpg
images/imagenet/006278.jpg
images/imagenet/011721.jpg
images/imagenet/023036.jpg
images/imagenet/003281.jpg
images/imagenet/018018.jpg
images/imagenet/007438.jpg
images/imagenet/002366.jpg
images/imagenet/015198.jpg
images/imagenet/019658.jpg
i

...

images/sun/011100.jpg
images/sun/026149.jpg
images/sun/023617.jpg
images/sun/003307.jpg
images/sun/006459.jpg
images/sun/022057.jpg
images/sun/010740.jpg
images/sun/019079.jpg
images/sun/007219.jpg
images/sun/002547.jpg
images/sun/002935.jpg
images/sun/002231.jpg
images/sun/022086.jpg
images/sun/010791.jpg
images/sun/002596.jpg
images/sun/022721.jpg
images/sun/015768.jpg
images/sun/010036.jpg
images/sun/026198.jpg
images/sun/003471.jpg
images/sun/003803.jpg
images/sun/011676.jpg
images/sun/014128.jpg
images/sun/023161.jpg
images/sun/006488.jpg
images/sun/000916.jpg
images/sun/000564.jpg
images/sun/012763.jpg
images/sun/020074.jpg
images/sun/009988.jpg
images/sun/004808.jpg
images/sun/001324.jpg
images/sun/021193.jpg
images/sun/013684.jpg
images/sun/001483.jpg
images/sun/021634.jpg
images/sun/013123.jpg
images/sun/018289.jpg
images/sun/003010.jpg
images/sun/023972.jpg
images/sun/023500.jpg
images/sun/014549.jpg
images/sun/011217.jpg
images/sun/002650.jpg
images/sun/022895.jpg
images/sun

images/sun/019847.jpg
images/sun/019435.jpg
images/sun/006015.jpg
images/sun/018275.jpg
images/sun/026505.jpg
images/sun/026977.jpg
images/sun/014212.jpg
images/sun/026273.jpg
images/sun/014564.jpg
images/sun/014916.jpg
images/sun/018971.jpg
images/sun/018503.jpg
images/sun/006763.jpg
images/sun/019343.jpg
images/sun/007684.jpg
images/sun/015324.jpg
images/sun/010808.jpg
images/sun/007123.jpg
images/sun/019896.jpg
images/sun/015483.jpg
images/sun/004791.jpg
images/sun/021278.jpg
images/sun/016231.jpg
images/sun/024954.jpg
images/sun/024526.jpg
images/sun/004036.jpg
images/sun/001768.jpg
images/sun/016596.jpg
images/sun/008051.jpg
images/sun/024281.jpg
images/sun/025366.jpg
images/sun/017803.jpg
images/sun/017471.jpg
images/sun/020438.jpg
images/sun/012488.jpg
images/sun/009611.jpg
images/sun/000128.jpg
images/sun/005676.jpg
images/sun/019722.jpg
images/sun/027052.jpg
images/sun/015745.jpg
images/sun/019085.jpg
images/sun/007930.jpg
images/sun/007542.jpg
images/sun/014105.jpg
images/sun

images/sun/019534.jpg
images/sun/006114.jpg
images/sun/018374.jpg
images/sun/014313.jpg
images/sun/026404.jpg
images/sun/026876.jpg
images/sun/008247.jpg
images/sun/016780.jpg
images/sun/024097.jpg
images/sun/004220.jpg
images/sun/016027.jpg
images/sun/008992.jpg
images/sun/013779.jpg
images/sun/024730.jpg
images/sun/004587.jpg
images/sun/005460.jpg
images/sun/005812.jpg
images/sun/009875.jpg
images/sun/020189.jpg
images/sun/009407.jpg
images/sun/000499.jpg
images/sun/025170.jpg
images/sun/012139.jpg
images/sun/017667.jpg
images/sun/006907.jpg
images/sun/006575.jpg
images/sun/018715.jpg
images/sun/026065.jpg
images/sun/014772.jpg
images/sun/015695.jpg
images/sun/027182.jpg
images/sun/007335.jpg
images/sun/002819.jpg
images/sun/015132.jpg
images/sun/007492.jpg
images/sun/019155.jpg
images/sun/023205.jpg
images/sun/011960.jpg
images/sun/011512.jpg
images/sun/026929.jpg
images/sun/003715.jpg
images/sun/014999.jpg
images/sun/010352.jpg
images/sun/022837.jpg
images/sun/022445.jpg
images/sun

images/sun/016990.jpg
images/sun/013969.jpg
images/sun/024552.jpg
images/sun/024920.jpg
images/sun/016245.jpg
images/sun/008782.jpg
images/sun/009665.jpg
images/sun/005602.jpg
images/sun/017405.jpg
images/sun/017877.jpg
images/sun/025312.jpg
images/sun/006717.jpg
images/sun/003049.jpg
images/sun/023559.jpg
images/sun/014962.jpg
images/sun/014510.jpg
images/sun/026207.jpg
images/sun/018577.jpg
images/sun/018905.jpg
images/sun/002609.jpg
images/sun/019490.jpg
images/sun/007157.jpg
images/sun/015885.jpg
images/sun/019337.jpg
images/sun/015350.jpg
images/sun/022319.jpg
images/sun/009204.jpg
images/sun/005263.jpg
images/sun/025773.jpg
images/sun/017064.jpg
images/sun/004851.jpg
images/sun/004423.jpg
images/sun/016183.jpg
images/sun/008444.jpg
images/sun/008836.jpg
images/sun/024694.jpg
images/sun/004384.jpg
images/sun/016624.jpg
images/sun/024133.jpg
images/sun/007536.jpg
images/sun/007944.jpg
images/sun/002268.jpg
images/sun/015096.jpg
images/sun/019756.jpg
images/sun/007291.jpg
images/sun

images/sun/011504.jpg
images/sun/023213.jpg
images/sun/003703.jpg
images/sun/024679.jpg
images/sun/013630.jpg
images/sun/021127.jpg
images/sun/001390.jpg
images/sun/013197.jpg
images/sun/021680.jpg
images/sun/001845.jpg
images/sun/001437.jpg
images/sun/004369.jpg
images/sun/020767.jpg
images/sun/012070.jpg
images/sun/025039.jpg
images/sun/005529.jpg
images/sun/000277.jpg
images/sun/017089.jpg
images/sun/023672.jpg
images/sun/011165.jpg
images/sun/003362.jpg
images/sun/010725.jpg
images/sun/022032.jpg
images/sun/002285.jpg
images/sun/010082.jpg
images/sun/022795.jpg
images/sun/002522.jpg
images/sun/002950.jpg
images/sun/011389.jpg
images/sun/006177.jpg
images/sun/003629.jpg
images/sun/023339.jpg
images/sun/014370.jpg
images/sun/026815.jpg
images/sun/026467.jpg
images/sun/018317.jpg
images/sun/002069.jpg
images/sun/007737.jpg
images/sun/015297.jpg
images/sun/007090.jpg
images/sun/019557.jpg
images/sun/019925.jpg
images/sun/015942.jpg
images/sun/015530.jpg
images/sun/022579.jpg
images/sun

In [6]:
# image datasest to be processed
name_dataset = 'sun+imagenetaval'
root_image = 'images'
with open('images/imagelist.txt') as f:
    lines = f.readlines()
imglist = []
for line in lines:
    line = line.rstrip()
    imglist.append(root_image + '/' + line)

In [7]:
features_blobs = []
def hook_feature(module, input, output):
    # hook the feature extractor
    features_blobs.append(np.squeeze(output.data.cpu().numpy()))

for name in features_names:
    model._modules.get(name).register_forward_hook(hook_feature)

In [8]:
# image transformer
tf = trn.Compose([
        trn.Resize(img_size),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [9]:
dataset = Dataset(imglist, tf)
loader = data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False)

In [10]:
# extract the max value activaiton for each image
imglist_results = []
maxfeatures = [None] * len(features_names)
num_batches = len(dataset) / batch_size
for batch_idx, (input, paths) in enumerate(loader):
    del features_blobs[:]
    print('%d / %d' % (batch_idx+1, num_batches))
    input = input.cuda()
    input_var = V(input, volatile=True)
    logit = model.forward(input_var)
    imglist_results = imglist_results + list(paths)
    if maxfeatures[0] is None:
        # initialize the feature variable
        for i, feat_batch in enumerate(features_blobs):
            size_features = (len(dataset), feat_batch.shape[1])
            maxfeatures[i] = np.zeros(size_features)
    start_idx = batch_idx*batch_size
    end_idx = min((batch_idx+1)*batch_size, len(dataset))
    for i, feat_batch in enumerate(features_blobs):
        maxfeatures[i][start_idx:end_idx] = np.max(np.max(feat_batch,3),2)

1 / 815
2 / 815


/home/saurabh/anaconda3/envs/py36torch/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':


3 / 815
4 / 815
5 / 815
6 / 815
7 / 815
8 / 815
9 / 815
10 / 815
11 / 815
12 / 815
13 / 815
14 / 815
15 / 815
16 / 815
17 / 815
18 / 815
19 / 815
20 / 815
21 / 815
22 / 815
23 / 815
24 / 815
25 / 815
26 / 815
27 / 815
28 / 815
29 / 815
30 / 815
31 / 815
32 / 815
33 / 815
34 / 815
35 / 815
36 / 815
37 / 815
38 / 815
39 / 815
40 / 815
41 / 815
42 / 815
43 / 815
44 / 815
45 / 815
46 / 815
47 / 815
48 / 815
49 / 815
50 / 815
51 / 815
52 / 815
53 / 815
54 / 815
55 / 815
56 / 815
57 / 815
58 / 815
59 / 815
60 / 815
61 / 815
62 / 815
63 / 815
64 / 815
65 / 815
66 / 815
67 / 815
68 / 815
69 / 815
70 / 815
71 / 815
72 / 815
73 / 815
74 / 815
75 / 815
76 / 815
77 / 815
78 / 815
79 / 815
80 / 815
81 / 815
82 / 815
83 / 815
84 / 815
85 / 815
86 / 815
87 / 815
88 / 815
89 / 815
90 / 815
91 / 815
92 / 815
93 / 815
94 / 815
95 / 815
96 / 815
97 / 815
98 / 815
99 / 815
100 / 815
101 / 815
102 / 815
103 / 815
104 / 815
105 / 815
106 / 815
107 / 815
108 / 815
109 / 815
110 / 815
111 / 815
112 / 815
113 

In [12]:
# generate the top activated images
output_folder = 'result_segments/%s' % model_name
if not os.path.exists(output_folder):
    os.makedirs(output_folder + '/image')

In [13]:
# output the html first
for layerID, layer in enumerate(features_names):
    file_html = os.path.join(output_folder, layer + '.html')
    with open(file_html, 'w') as f:
        num_units = maxfeatures[layerID].shape[1]
        lines_units = ['%s-unit%03d.jpg' % (layer, unitID) for unitID in range(num_units)]
        lines_units = ['unit%03d<br><img src="image/%s">'%(unitID, lines_units[unitID]) for unitID in range(num_units)]
        f.write('\n<br>'.join(lines_units))

In [14]:
# generate the unit visualization
for layerID, layer in enumerate(features_names):
    num_units = maxfeatures[layerID].shape[1]
    imglist_sorted = []
    # load the top actiatied image list into one list
    for unitID in range(num_units):
        activations_unit = np.squeeze(maxfeatures[layerID][:, unitID])
        idx_sorted = np.argsort(activations_unit)[::-1]
        imglist_sorted += [imglist[item] for item in idx_sorted[:num_top]]

    # data loader for the top activated images
    loader_top = data.DataLoader(
        Dataset(imglist_sorted, tf),
        batch_size=num_top,
        num_workers=num_workers,
        shuffle=False)
    for unitID, (input, paths) in enumerate(loader_top):
        del features_blobs[:]
        print('%d / %d' % (unitID+1, num_units))
        input = input.cuda()
        input_var = V(input, volatile=True)
        logit = model.forward(input_var)
        feature_maps = features_blobs[layerID]

        images_input = input.cpu().numpy()
        max_value = 0
        output_unit = []
        for i in range(num_top):
            feature_map = feature_maps[i][unitID]
            if max_value == 0:
                max_value = np.max(feature_map)
            feature_map = feature_map / max_value
            mask = cv2.resize(feature_map, segment_size)
            mask[mask < threshold_scale] = 0.0 # binarize the mask
            mask[mask > threshold_scale] = 1.0

            img = cv2.imread(paths[i])
            img = cv2.resize(img, segment_size)
            img = cv2.normalize(img.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
            img_mask = np.multiply(img, mask[:,:, np.newaxis])
            img_mask = np.uint8(img_mask * 255)
            output_unit.append(img_mask)
            output_unit.append(np.uint8(np.ones((segment_size[0],margin,3))*255))
        montage_unit = np.concatenate(output_unit, axis=1)
        cv2.imwrite(os.path.join(output_folder, 'image', '%s-unit%03d.jpg'%(layer, unitID)), montage_unit)
print('done check results in ' + output_folder)

1 / 512
2 / 512
3 / 512
4 / 512
5 / 512
6 / 512
7 / 512
8 / 512


/home/saurabh/anaconda3/envs/py36torch/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


9 / 512
10 / 512
11 / 512
12 / 512
13 / 512
14 / 512
15 / 512
16 / 512
17 / 512
18 / 512
19 / 512
20 / 512
21 / 512
22 / 512
23 / 512
24 / 512
25 / 512
26 / 512
27 / 512
28 / 512
29 / 512
30 / 512
31 / 512
32 / 512
33 / 512
34 / 512
35 / 512
36 / 512
37 / 512
38 / 512
39 / 512
40 / 512
41 / 512
42 / 512
43 / 512
44 / 512
45 / 512
46 / 512
47 / 512
48 / 512
49 / 512
50 / 512
51 / 512
52 / 512
53 / 512
54 / 512
55 / 512
56 / 512
57 / 512
58 / 512
59 / 512
60 / 512
61 / 512
62 / 512
63 / 512
64 / 512
65 / 512
66 / 512
67 / 512
68 / 512
69 / 512
70 / 512
71 / 512
72 / 512
73 / 512
74 / 512
75 / 512
76 / 512
77 / 512
78 / 512
79 / 512
80 / 512
81 / 512
82 / 512
83 / 512
84 / 512
85 / 512
86 / 512
87 / 512
88 / 512
89 / 512
90 / 512
91 / 512
92 / 512
93 / 512
94 / 512
95 / 512
96 / 512
97 / 512
98 / 512
99 / 512
100 / 512
101 / 512
102 / 512
103 / 512
104 / 512
105 / 512
106 / 512
107 / 512
108 / 512
109 / 512
110 / 512
111 / 512
112 / 512
113 / 512
114 / 512
115 / 512
116 / 512
117 / 512
11